In [1]:
from importlib.util import find_spec
%load_ext autoreload
%autoreload 2

%matplotlib inline

if find_spec("qml_hep_lhc") is None:
    import sys
    sys.path.append('../..')

In [2]:
import random
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt

from qml_hep_lhc.data import ElectronPhoton, MNIST, QuarkGluon
from qml_hep_lhc.models import QCNNHybrid
from tensorflow.keras.callbacks import ReduceLROnPlateau
import argparse
import numpy as np
from qml_hep_lhc.data.utils import tf_ds_to_numpy, create_tf_ds
from qml_hep_lhc.layers import QConv2D

import matplotlib.pyplot as plt
import time

2022-10-01 15:48:43.367654: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-01 15:48:43.367742: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-10-01 15:49:49.506184: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-10-01 15:49:49.506276: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (bhagvada): /proc/driver/nvidia/version does not exist
2022-10-01 15:49:49.531550: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
T

In [3]:
args = argparse.Namespace()

# Data
args.center_crop = 0.2
# args.resize = [8,8]
# args.normalize = 1
# args.binary_data = [0,1]
args.batch_size = 128
args.validation_split = 0.05
args.dataset_type = '3'
# args.labels_to_categorical = 1
args.opt = 'Ranger'

# Base Model
args.learning_rate = 1e-3
args.epochs = 50

# Quantum CNN Parameters
args.n_qubits = 1
args.n_layers = 1
args.ansatz = "NQubit"
args.sparse = 1

In [4]:
data = ElectronPhoton(args)
data.prepare_data()
data.setup()
print(data)

Center cropping...


2022-10-01 15:54:52.540237: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1638400000 exceeds 10% of free system memory.


Center cropping...

Dataset :Electron Photon 3
╒════════╤═══════════════════╤══════════════════╤══════════════════╤═══════════╕
│ Data   │ Train size        │ Val size         │ Test size        │ Dims      │
╞════════╪═══════════════════╪══════════════════╪══════════════════╪═══════════╡
│ X      │ (380000, 8, 8, 1) │ (20000, 8, 8, 1) │ (98000, 8, 8, 1) │ (8, 8, 1) │
├────────┼───────────────────┼──────────────────┼──────────────────┼───────────┤
│ y      │ (380000,)         │ (20000,)         │ (98000,)         │ (1,)      │
╘════════╧═══════════════════╧══════════════════╧══════════════════╧═══════════╛

╒══════════════╤═══════╤═══════╤════════╤═══════╤══════════════════════════╕
│ Type         │   Min │   Max │   Mean │   Std │ Samples for each class   │
╞══════════════╪═══════╪═══════╪════════╪═══════╪══════════════════════════╡
│ Train Images │     0 │  1.48 │   0.02 │  0.09 │ [190000, 190000]         │
├──────────────┼───────┼───────┼────────┼───────┼──────────────────────────┤


In [5]:
data.config()

{'input_dims': (8, 8, 1), 'output_dims': (1,), 'mapping': [0, 1]}

In [6]:
x_train, y_train =  tf_ds_to_numpy(data.train_ds) 
del data.train_ds

x_val, y_val =  tf_ds_to_numpy(data.val_ds) 
del data.val_ds

x_test, y_test =  tf_ds_to_numpy(data.test_ds) 
del data.test_ds

print(x_train.shape, x_val.shape, x_test.shape)

(380000, 8, 8, 1) (20000, 8, 8, 1) (98000, 8, 8, 1)


In [7]:
def make_pairs(x, y):
    """Creates a tuple containing image pairs with corresponding label.
    """
    
    num_classes = len(data.config()['mapping'])
    digit_indices = [np.where(y == i)[0] for i in range(num_classes)]
    pairs = []
    labels = []

    for idx1 in range(len(x)):
        # add a matching example
        x1 = x[idx1]
        label1 = int(y[idx1])
        idx2 = random.choice(digit_indices[label1])
        x2 = x[idx2]

        pairs += [[x1, x2]]
        labels += [0]

        # add a non-matching example
        label2 = random.randint(0, num_classes - 1)
        while label2 == label1:
            label2 = random.randint(0, num_classes - 1)

        idx2 = random.choice(digit_indices[label2])
        x2 = x[idx2]

        pairs += [[x1, x2]]
        labels += [1]

    return np.array(pairs), np.array(labels).astype("float32")


# make train pairs
pairs_train, labels_train = make_pairs(x_train, y_train)
del x_train, y_train
# make validation pairs
pairs_val, labels_val = make_pairs(x_val, y_val)
del x_val, y_val
# make test pairs
pairs_test, labels_test = make_pairs(x_test, y_test)
del x_test, y_test

In [8]:
print(pairs_train.shape, pairs_val.shape, pairs_test.shape)
print(labels_train.shape, labels_val.shape, labels_test.shape)

(760000, 2, 8, 8, 1) (40000, 2, 8, 8, 1) (196000, 2, 8, 8, 1)
(760000,) (40000,) (196000,)


In [9]:
x_train_1 = pairs_train[:, 0]  # x_train_1.shape is (60000, 28, 28)
x_train_2 = pairs_train[:, 1]

del pairs_train

x_val_1 = pairs_val[:, 0]  # x_val_1.shape = (60000, 28, 28)
x_val_2 = pairs_val[:, 1]

del pairs_val

x_test_1 = pairs_test[:, 0]  # x_test_1.shape = (20000, 28, 28)
x_test_2 = pairs_test[:, 1]

del pairs_test

In [10]:
def visualize(pairs, labels, to_show=6, num_col=3, predictions=None, test=False):
    """Creates a plot of pairs and labels, and prediction if it's test dataset.

    Arguments:
        pairs: Numpy Array, of pairs to visualize, having shape
               (Number of pairs, 2, 28, 28).
        to_show: Int, number of examples to visualize (default is 6)
                `to_show` must be an integral multiple of `num_col`.
                 Otherwise it will be trimmed if it is greater than num_col,
                 and incremented if if it is less then num_col.
        num_col: Int, number of images in one row - (default is 3)
                 For test and train respectively, it should not exceed 3 and 7.
        predictions: Numpy Array of predictions with shape (to_show, 1) -
                     (default is None)
                     Must be passed when test=True.
        test: Boolean telling whether the dataset being visualized is
              train dataset or test dataset - (default False).

    Returns:
        None.
    """

    # Define num_row
    # If to_show % num_col != 0
    #    trim to_show,
    #       to trim to_show limit num_row to the point where
    #       to_show % num_col == 0
    #
    # If to_show//num_col == 0
    #    then it means num_col is greater then to_show
    #    increment to_show
    #       to increment to_show set num_row to 1
    num_row = to_show // num_col if to_show // num_col != 0 else 1

    # `to_show` must be an integral multiple of `num_col`
    #  we found num_row and we have num_col
    #  to increment or decrement to_show
    #  to make it integral multiple of `num_col`
    #  simply set it equal to num_row * num_col
    to_show = num_row * num_col

    # Plot the images
    fig, axes = plt.subplots(num_row, num_col, figsize=(5, 5))
    for i in range(to_show):

        # If the number of rows is 1, the axes array is one-dimensional
        if num_row == 1:
            ax = axes[i % num_col]
        else:
            ax = axes[i // num_col, i % num_col]

        ax.imshow(tf.concat([pairs[i][0], pairs[i][1]], axis=1), cmap="gray")
        ax.set_axis_off()
        if test:
            ax.set_title("True: {} | Pred: {:.5f}".format(labels[i], predictions[i][0]))
        else:
            ax.set_title("Label: {}".format(labels[i]))
    if test:
        plt.tight_layout(rect=(0, 0, 1.9, 1.9), w_pad=0.0)
    else:
        plt.tight_layout(rect=(0, 0, 1.5, 1.5))
    plt.show()

In [11]:
visualize(pairs_train[:-1], labels_train[:-1], to_show=5, num_col=5)

NameError: name 'pairs_train' is not defined

In [12]:
# Provided two tensors t1 and t2
# Euclidean distance = sqrt(sum(square(t1-t2)))
def euclidean_distance(vects):
    """Find the Euclidean distance between two vectors.

    Arguments:
        vects: List containing two tensors of same length.

    Returns:
        Tensor containing euclidean distance
        (as floating point value) between vectors.
    """

    x, y = vects
    sum_square = tf.math.reduce_sum(tf.math.square(x - y), axis=1, keepdims=True)
    return tf.math.sqrt(tf.math.maximum(sum_square, tf.keras.backend.epsilon()))


input = layers.Input((8, 8, 1))

# model = QCNNHybrid(data.config(), args)
# x = model(input)
# x = layers.Flatten()(x)

input = layers.Input((8, 8, 1))
x = tf.keras.layers.BatchNormalization()(input)
x = layers.Conv2D(4, (3, 3), padding='same', activation="tanh")(x)
x = layers.AveragePooling2D(pool_size=(2, 2))(x)
x = layers.Conv2D(16, (3, 3), padding='same', activation="tanh")(x)
x = layers.AveragePooling2D(pool_size=(2, 2))(x)
x = layers.Flatten()(x)

x = tf.keras.layers.BatchNormalization()(x)
x = layers.Dense(2, activation="tanh")(x)
embedding_network = keras.Model(input, x)


input_1 = layers.Input((8, 8, 1))
input_2 = layers.Input((8, 8, 1))

# As mentioned above, Siamese Network share weights between
# tower networks (sister networks). To allow this, we will use
# same embedding network for both tower networks.
tower_1 = embedding_network(input_1)
tower_2 = embedding_network(input_2)

merge_layer = layers.Lambda(euclidean_distance)([tower_1, tower_2])
normal_layer = tf.keras.layers.BatchNormalization()(merge_layer)
output_layer = layers.Dense(1, activation="sigmoid")(normal_layer)
siamese = keras.Model(inputs=[input_1, input_2], outputs=output_layer)

In [13]:
def loss(margin=1):
    """Provides 'constrastive_loss' an enclosing scope with variable 'margin'.

    Arguments:
        margin: Integer, defines the baseline for distance for which pairs
                should be classified as dissimilar. - (default is 1).

    Returns:
        'constrastive_loss' function with data ('margin') attached.
    """

    # Contrastive loss = mean( (1-true_value) * square(prediction) +
    #                         true_value * square( max(margin-prediction, 0) ))
    def contrastive_loss(y_true, y_pred):
        """Calculates the constrastive loss.

        Arguments:
            y_true: List of labels, each label is of type float32.
            y_pred: List of predictions of same length as of y_true,
                    each label is of type float32.

        Returns:
            A tensor containing constrastive loss as floating point value.
        """

        square_pred = tf.math.square(y_pred)
        margin_square = tf.math.square(tf.math.maximum(margin - (y_pred), 0))
        return tf.math.reduce_mean(
            (1 - y_true) * square_pred + (y_true) * margin_square
        )

    return contrastive_loss

In [14]:
margin = 1
siamese.compile(loss=loss(margin=margin), optimizer="RMSprop", metrics=["accuracy"])
siamese.summary(expand_nested=True)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 8, 8, 1)]    0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 8, 8, 1)]    0           []                               
                                                                                                  
 model (Functional)             (None, 2)            1022        ['input_3[0][0]',                
                                                                  'input_4[0][0]']                
|¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯¯|
| input_2 (InputLayer)         [(None, 8, 8, 1)]    0           []                          

In [ ]:
history = siamese.fit(
    [x_train_1, x_train_2],
    labels_train,
    validation_data=([x_val_1, x_val_2], labels_val),
    batch_size=args.batch_size,
    epochs=args.epochs,
)

Epoch 1/50
2837/5938 [=============>................] - ETA: 29s - loss: 0.2539 - accuracy: 0.4977